In [ ]:
import json
import re
import os
import sqlite3
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify, make_response
from functools import wraps
from google.cloud import datastore

In [ ]:
# 챗봇에서는 동작하고 있는 강의실 모델을 동작시키는 코드입니다.
# 노트북에서는 다른 루트로 동작합니다.
# 필요하다면 올려둔 classroom_model.ipynb 파일의 내용으로 대체해도 됩니다.
%run classroom_model.ipynb

In [ ]:
# 챗봇에서는 동작하고 있는 라운지 모델을 동작시키는 코드입니다.
# 노트북에서는 다른 루트로 동작합니다.
# 필요하다면 올려둔 lounge_mlp_model.ipynb 파일의 내용으로 대체해도 됩니다.
%run lounge_mlp_model.ipynb

In [ ]:
app = Flask(__name__)

# 인증
def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwards):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            return authenticate()
        return f(*args, **kwards)
    return decorated

def check_auth(username, password):
    """This function is called to check if a username /
    password combination is valid.
    """       
    uname="admin@ppp.ppp"
    pwd="admin"
    return username == uname and password == pwd
    
def authenticate():
    """Sends a 401 response that enables basic auth"""
    logging.info("inside authenticate")
    return Response(
    'Could not verify your access level for that URL.\n'
    'You have to login with proper credentials', 401,
    {'WWW-Authenticate': 'Basic realm="Login Required"'})
    
    

# main
@app.route('/webhook/', methods=['POST'])
@requires_auth


def handle():
    req = request.get_json(silent=True, force=True)
    print('Request:')
    print(json.dumps(req, indent=4))
    if req.get('queryResult').get('action') != 'lookup':
      return {'테스트에 실패하였습니다. 오류 : 액션 감지 실패'}
    
    action = req['queryResult']['action']
    
    print('action :', action)
    
    session_id = req['session']
    
    """action == 'a' 에서 a 는 각 intent의 호출된 action으로 어느 intent인지 구별하는 과정입니다.
    각 intent별로 다르게 입력된 변수들을 추출하여 이를 output 변수를 만드는 함수를 호출하고 있습니다."""
    
    if action == 'ClassroomRec':
        landmark = req.get('queryResult').get('parameters').get('nearby-landmark')
        start = req.get('queryResult').get('parameters').get('time-period').get('startTime')
        end = req.get('queryResult').get('parameters').get('time-period').get('endTime')
        
        if req.get('queryResult').get('parameters').get('number') == '':
            number = 1
        else:
            number = int(req.get('queryResult').get('parameters').get('number'))
          
        if req.get('queryResult').get('parameters').get('power-socket-preference') == '':
            power_socket = 3
        else:
            power_socket = int(req.get('queryResult').get('parameters').get('power-socket-preference'))
            
        if req.get('queryResult').get('parameters').get('table-preference') == '':
            table_type = 1
        else:
            table_type = int(req.get('queryResult').get('parameters').get('table-preference'))
            
        if req.get('queryResult').get('parameters').get('chair-preference') == '':
            chair_type = 0
        else:
            chair_type = int(req.get('queryResult').get('parameters').get('chair-preference'))
        
        if landmark == '':
            landmark = '정경관'
        
        print('number :', number, 'landmark :', landmark, 'start :', start, 'end :', end)
        
        rsp = getClassroom(session_id, number, landmark, start, end, power_socket, table_type, chair_type)
        
    elif action == 'CampusRec':
        landmark = req.get('queryResult').get('parameters').get('nearby-landmark')
        
        power_socket = int(req.get('queryResult').get('parameters').get('power-socket'))
        table_type = int(req.get('queryResult').get('parameters').get('table-type'))
        facility = int(req.get('queryResult').get('parameters').get('facility'))
        computer = int(req.get('queryResult').get('parameters').get('computer'))
        
        if req.get('queryResult').get('parameters').get('number') == '':
            number = 1
        else:
            number = int(req.get('queryResult').get('parameters').get('number'))
        
        start = req.get('queryResult').get('parameters').get('time-period').get('startTime')
        end = req.get('queryResult').get('parameters').get('time-period').get('endTime')
        
        theme = req.get('queryResult').get('parameters').get('theme')
        if theme == '공부':
            theme = 0
        elif theme == '휴식':
            theme = 1
        elif theme == '대화':
            theme = 2
        elif theme == '팀공부':
            theme = 3
        else:
            theme = 0
        
        rsp = getCampus(landmark, number, power_socket, table_type, facility, theme, start, computer)
        
    # 강의실 예약 인텐트
    elif action == 'RecommendationIntent.RecommendationIntent-custom':
        list_index = int(req.get('queryResult').get('parameters').get('list-index'))
        output_context = req.get('queryResult').get('outputContexts')[0]
        member_num = int(output_context.get('parameters').get('number'))
        starttime = output_context.get('parameters').get('time-period').get('startTime')
        endtime = output_context.get('parameters').get('time-period').get('endTime')
        rsp = makeClassroomReservation(session_id, list_index, member_num, starttime, endtime)
        
    #다른 추천결과를 보여줄 때(추후 확장할 기능) 
    elif action == 'AnotherRec':
        rsp = getAnother()
    
    
    #결과를 가공하고 챗봇으로 return
    print(rsp)
    rsp = json.dumps(rsp, indent=4)
    print(rsp)
    
    r = make_response(rsp)
    r.headers['Content-Type'] = 'application/json'
    return r


"""각 intent별 handler입니다. 입력된 변수를 통해 결과를 생성하는 함수들입니다.
모델을 거쳐 나온 변수들은 return bracket form에 맞추는 buildReply 함수로 전해집니다. 이 결과를 리턴합니다.""" 
def getClassroom(session_id, number, landmark, start, end, power_socket, table_type, chair_type):
    building_name_list = classroom_Top9(number, (start, end), landmark, power_socket, table_type, chair_type)
    
    recs = []

    #DB 호출 (추천 내역 저장)
    conn = sqlite3.connect("gmc.db")
    cur = conn.cursor()
    query_str = str('"' + building_name_list[0][0]) + '"' + ', ' + '"' + str(building_name_list[0][1])+ '"'
    for i, tup in enumerate(building_name_list):
        if i != 0:
            query_str += (', ' + '"' + tup[0] + '"' + ', ' + '"' + tup[1] + '"')
        recs.append(tup[0] + ' ' + tup[1])
    cur.execute('INSERT INTO recommendation_history VALUES (' + '"' + session_id + '"' + ', 0, ' + query_str + ');')
    conn.commit()
    conn.close()
    
    #응답 생성
    #return {'fulfillmentText': (period)에 (number)명이 이용 가능한 강의실은 다음과 같습니다. (landmark)'}
    return buildReply(recs)


def getCampus(landmark, number, power_socket, table_type, facility, theme, time, computer):
    recs = lounge_Top9(theme, landmark, time, number, table_type, power_socket, computer, facility)
    print(recs)
    # return {'fulfillmentText': '(landmark) 주변에서 이용 가능한 교내 공간은 다음과 같습니다.'}
    return buildCampusReply(recs)
    

def makeClassroomReservation(session_id, list_index, member_num, starttime, endtime):
    #DB 호출 (추천 내역 확인)
    conn = sqlite3.connect("gmc.db")
    cur = conn.cursor()
    cur.execute("SELECT * FROM recommendation_history WHERE session_id = '" + session_id + "';")
    rows = cur.fetchall()
    if len(rows) == 1:
        row = rows[0]
    else:
        row = rows[-1]
    building_name_list = []
    for i in range(2, 20, 2):
        building_name_list.append((row[i], row[i+1]))
    print(building_name_list)
    res_building = building_name_list[list_index-1][0]
    res_name = building_name_list[list_index-1][1]
    #예약
    cur.execute('INSERT INTO classroom_reservation VALUES (' +
        '"' + session_id + '"' + ', ' +
        '"' + res_building + '"' + ', ' +
        '"' + res_name + '"' + ', ' +
        str(member_num) + ', ' +
        '"' + starttime + '"' + ', ' +
        '"' + endtime + '"' + ');')

    conn.commit()
    conn.close()
    return buildReservationReply(res_building, res_name)


nl = '\n'

"""response 형식에 맞도록 답변을 가공해주는 함수입니다."""
def buildReply(recs):
    return {
        "fulfillmentMessages": [
      {
        "text": {
          "text": [
            "추천하는 강의실은 다음과 같습니다 " + nl +
              "1. " + recs[0] + nl +
              "2. " + recs[1] + nl +
              "3. " + recs[2] + nl +
              "4. " + recs[3] + nl +
              "5. " + recs[4] + nl +
              "6. " + recs[5] + nl +
              "7. " + recs[6] + nl +
              "8. " + recs[7] + nl +
              "9. " + recs[8] + nl + "어느 것을 예약해드릴까요?"
          ]
        }
      }
    ]
    }
def rec2str(rec):
  return rec[0] + " " + rec[1]
def buildCampusReply(recs):
    return {
        "fulfillmentMessages": [
      {
        "text": {
          "text": [
            "추천하는 라운지는 다음과 같습니다 " + nl +
              "1. " + rec2str(recs[0]) + nl +
              "2. " + rec2str(recs[1]) + nl +
              "3. " + rec2str(recs[2]) + nl +
              "4. " + rec2str(recs[3]) + nl +
              "5. " + rec2str(recs[4]) + nl +
              "6. " + rec2str(recs[5]) + nl +
              "7. " + rec2str(recs[6]) + nl +
              "8. " + rec2str(recs[7]) + nl +
              "9. " + rec2str(recs[8]) + nl
          ]
        }
      }
    ]
    }


def buildReservationReply(res_building, res_name):
    return {
        "fulfillmentMessages": [
      {
        "text": {
          "text": [
            "강의실 예약이 완료되었습니다" + nl +
              "건물 : " + res_building + nl +
              "강의실 : " + res_name
          ]
        }
      }
    ]
    }

#default Falsk form
if __name__ == '__main__':
    app.run(host='0.0.0.0')